In [11]:
import pandas as pd

df = pd.read_excel('SIB-200 languages - ACL.xlsx')
df

,Folder Name,Language Name,ISO 639 - 1,ISO 639 - 2 (B),ISO 639 - 2 (T),ISO 639 - 3,Script (ISO 15924),Language Family,Population,Language Vitality,...,XLM-R Train Tokens,F1 Bloom-560M sampling,F1 Bloom-560M beam-search,F1 PolyLM-1.7b sampling,F1 PolyLM-1.7b beam-search,F1 Bloomz-560M,F1 Bloomz-1b1,F1 Bloomz-1b7,F1 Bloomz-3b,F1 Bloomz-7b1
0,ace_Arab,Aceh,NaN,ace,ace,ace,Arab,Austronesian,1 million to 1 billion,Endangered,...,NaN,0.035714,0.014548,0.042353,0.046569,0.105192,0.094892,0.114471,0.142853,0.1020
1,ace_Latn,Aceh,NaN,ace,ace,ace,Latn,Austronesian,1 million to 1 billion,Endangered,...,NaN,0.029306,0.074956,0.109695,0.116462,0.352888,0.260779,0.262736,0.335329,0.4112
2,acm_Arab,Mesopotamian Spoken Arabic,NaN,NaN,NaN,acm,Arab,Afro-Asiatic,1 million to 1 billion,Institutional,...,NaN,0.053613,0.036458,0.068885,0.128984,0.459507,0.382058,0.335226,0.435668,0.4919
3,acq_Arab,"Arabic, Ta’izzi-Adeni Spoken",NaN,NaN,NaN,acq,Arab,Afro-Asiatic,1 million to 1 billion,Institutional,...,NaN,0.038295,0.038158,0.049175,0.080324,0.460365,0.330938,0.353313,0.453236,0.4965
4,aeb_Arab,Tunisian Spoken Arabic,NaN,NaN,NaN,aeb,Arab,Afro-Asiatic,1 million to 1 billion,Institutional,...,NaN,0.056647,0.054823,0.089042,0.068235,0.455299,0.343693,0.315781,0.442202,0.4663
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,yue_Hant,Yue Chinese,NaN,NaN,NaN,yue,Hant,Sino-Tibetan,1 million to 1 billion,Institutional,...,NaN,0.110311,0.031746,0.067612,0.057919,0.344580,0.321814,0.366669,0.482894,0.4993
200,zho_Hans,Chinese,zh,chi,zho,zho,Hans,Sino-Tibetan,1 billion plus,Institutional,...,259.0,0.068555,0.032927,0.075991,0.076000,0.327759,0.330228,0.391396,0.489759,0.5081
201,zho_Hant,Chinese,zh,chi,zho,zho,Hant,Sino-Tibetan,1 billion plus,Institutional,...,176.0,0.036136,0.051013,0.128679,0.090060,0.337263,0.356543,0.376002,0.485621,0.5203
202,zsm_Latn,Standard Malay,ms,NaN,NaN,zsm,Latn,Austronesian,None,Institutional,...,1318.0,0.067582,0.047711,0.072207,0.090327,0.392790,0.316372,0.298548,0.451145,0.4952


In [15]:
import pandas as pd

# Select relevant columns for analysis
columns_to_plot = ['Language Name', 'Language Family', 'Script (ISO 15924)', 'Resource Level', 'Bloom Train Data Percentage', 'BLOOMZ Finetune Data', 
                   'F1 Bloomz-560M', 'F1 Bloomz-1b1', 'F1 Bloomz-1b7', 'F1 Bloomz-3b', 'F1 Bloomz-7b1']

# Melt the DataFrame to make it suitable for plotting
melted_df = pd.melt(df, id_vars=columns_to_plot[:6], value_vars=columns_to_plot[6:],
                    var_name='Model', value_name='F1 Score')
melted_df

,Language Name,Language Family,Script (ISO 15924),Resource Level,Bloom Train Data Percentage,BLOOMZ Finetune Data,Model,F1 Score
0,Aceh,Austronesian,Arab,1,NaN,NaN,F1 Bloomz-560M,0.105192
1,Aceh,Austronesian,Latn,1,NaN,NaN,F1 Bloomz-560M,0.352888
2,Mesopotamian Spoken Arabic,Afro-Asiatic,Arab,None,NaN,NaN,F1 Bloomz-560M,0.459507
3,"Arabic, Ta’izzi-Adeni Spoken",Afro-Asiatic,Arab,None,NaN,NaN,F1 Bloomz-560M,0.460365
4,Tunisian Spoken Arabic,Afro-Asiatic,Arab,None,NaN,NaN,F1 Bloomz-560M,0.455299
...,...,...,...,...,...,...,...,...
1015,Yue Chinese,Sino-Tibetan,Hant,None,NaN,NaN,F1 Bloomz-7b1,0.499300
1016,Chinese,Sino-Tibetan,Hans,5,16.200,4.51,F1 Bloomz-7b1,0.508100
1017,Chinese,Sino-Tibetan,Hant,5,0.050,NaN,F1 Bloomz-7b1,0.520300
1018,Standard Malay,Austronesian,Latn,3,NaN,NaN,F1 Bloomz-7b1,0.495200


In [16]:
import plotly.io as pio
pio.renderers.default = 'chrome'

In [68]:
from functools import reduce
import operator

def filter_df(melted_df: pd.DataFrame, filter: str = None, value: str = None, only_bloom: bool = False, only_bloomz: bool = False) -> pd.DataFrame:
    conditions = []  # Initialize an empty list to store conditions
    if filter is not None and value is not None:
        conditions.append(melted_df[filter] == value)
    if only_bloom:
        conditions.append(melted_df['Bloom Train Data Percentage'] > 0)
    if only_bloomz:
        conditions.append(melted_df['BLOOMZ Finetune Data'] > 0)

    # If no conditions were added, return the original DataFrame
    if not conditions:
        return melted_df

    # Use reduce with the logical AND operator to combine conditions
    combined_conditions = reduce(operator.and_, conditions)

    return melted_df[combined_conditions]

In [71]:
import plotly.express as px
only_bloom = False
only_bloomz = False
filter = None
value = None

filtered_df = filter_df(melted_df, only_bloom=only_bloom, only_bloomz=only_bloomz)
fig = px.box(filtered_df, x='Model', y='F1 Score',
             color='Model', title=f'Box Plot for F1 Scores',
             labels={'F1 Score': 'F1 Score', 'Model': 'Model'})

fig.show()

In [41]:
import plotly.express as px

only_bloomz = False
only_bloom = False
filter = 'Resource Level'

for value in df[filter].unique():
    filtered_df = filter_df(melted_df, filter, value, only_bloom, only_bloomz)
    fig = px.box(filtered_df, x='Model', y='F1 Score',
                 color='Model', title=f'Box Plot for F1 Scores by Model ({filter} = {value}, Only Bloomz = {only_bloomz}, Only Bloom = {only_bloom})',
                 labels={'F1 Score': 'F1 Score', 'Model': 'Model'})
    fig.show()
# Create a scatter plot using Plotly Express
# fig = px.scatter(melted_df, x='Language Name', y='F1 Score', color='Model',
#                  facet_col='Language Family', facet_col_wrap=3,
#                  labels={'F1 Score': 'F1 Score', 'Language Name': 'Language Name'},
#                  title='F1 Score by Language and Model')
# 
# # Customize layout if needed
# fig.update_layout(height=2000, width=1200)
# 
# # Show the plot
# fig.show()

In [42]:
import plotly.express as px
import warnings

warnings.filterwarnings('ignore')

# Radar chart for all models in the same plot using plotly
only_bloomz = False
only_bloom = False
filter = 'Language Family'

for value in df[filter].unique():
    filtered_df = filter_df(melted_df, filter, value, only_bloom, only_bloomz)
    fig = px.line_polar(filtered_df, r='F1 Score', theta='Language Name', line_close=True,
                        color='Model', title=f'Radar Chart for F1 Scores by Language and Model ({value})',
                        height=600, width=800)
    fig.show()

In [24]:
melted_df[(melted_df[filter] == value) & (melted_df['Bloom Train Data Percentage'] > 0 | (melted_df['BLOOMZ Finetune Data'] > 0))]

,Language Name,Language Family,Script (ISO 15924),Resource Level,Bloom Train Data Percentage,BLOOMZ Finetune Data,Model,F1 Score
200,Chinese,Sino-Tibetan,Hans,5,16.2,4.51,F1 Bloomz-560M,0.327759
404,Chinese,Sino-Tibetan,Hans,5,16.2,4.51,F1 Bloomz-1b1,0.330228
608,Chinese,Sino-Tibetan,Hans,5,16.2,4.51,F1 Bloomz-1b7,0.391396
812,Chinese,Sino-Tibetan,Hans,5,16.2,4.51,F1 Bloomz-3b,0.489759
1016,Chinese,Sino-Tibetan,Hans,5,16.2,4.51,F1 Bloomz-7b1,0.508100
